# Characterize the orbit of the lense observed
author: [Mathieu Renzo](mrenzo@flatironinstitute.org)

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

import gala as ga

import sys
sys.path.append("../src")
from kicks import integrate_orbits_with_kicks

import socket
if socket.gethostname() == "ccalin010.flatironinstitute.org":
    sys.path.append('/mnt/home/mrenzo/codes/python_stuff/plotFunc/')
    from plotDefaults import * #set_plot_defaults_from_matplotlibrc
elif socket.gethostname() == 't490s':
    sys.path.append('/home/math/Documents/Research/codes/plotFunc/')
    from plotDefaults import * #set_plot_defaults_from_matplotlibrc
# to draw kicks from Maxwellians
from scipy.stats import norm
import random
import scipy.optimize
from scipy.stats import maxwell, uniform
from tqdm import tqdm

In [ ]:
if socket.gethostname() == "ccalin010.flatironinstitute.org":
    set_plot_defaults_from_matplotlibrc("/mnt/home/mrenzo/codes/python_stuff/plotFunc/")
elif socket.gethostname() == 't490s':
    set_plot_defaults_from_matplotlibrc('/home/math/Documents/Research/codes/plotFunc/')

In [ ]:
# define reference frame
galcen_frame = coord.Galactocentric()
# define max time and num timesteps
t_max = 1*u.Gyr
n_steps= 1000
pot = gp.MilkyWayPotential()
w0s = []
for rv in [0,0]: 
    c = coord.SkyCoord(
        ra="17:51:40.2082",
        dec="-29:53:26.502",
        unit=(u.hourangle, u.degree),
        distance=1.58*u.kpc,
        pm_ra_cosdec=-4.36*u.mas/u.yr,# * cosdec,
        pm_dec=3.06*u.mas/u.yr,
        radial_velocity=rv*u.km/u.s
    )

    w0 = gd.PhaseSpacePosition(c.transform_to(galcen_frame).data)
    w0s.append(w0)
    
w0s = gd.combine(w0s)

orbits = pot.integrate_orbit(w0s, dt=t_max / n_steps, n_steps=n_steps)

In [ ]:
# Eccentricity distribution of orbits compatible with BH observed position and velocity
n_orbits = 1000
# sample
rvs = np.linspace(-100,100, n_orbits)
weights = uniform.rvs(size=n_orbits)
# Mathieu: rv seems not to matter: is this because it is towards the GC?
w0s = []
for rv in tqdm(rvs): 
    c = coord.SkyCoord(
        ra="17:51:40.2082",
        dec="-29:53:26.502",
        unit=(u.hourangle, u.degree),
        distance=1.58*u.kpc,
        pm_ra_cosdec=-4.36*u.mas/u.yr,# * cosdec,
        pm_dec=3.06*u.mas/u.yr,
        radial_velocity=rv*u.km/u.s
    )

    w0 = gd.PhaseSpacePosition(c.transform_to(galcen_frame).data)
    w0s.append(w0)
    
w0s = gd.combine(w0s)

no_kicks = integrate_orbits_with_kicks(pot, w0s, dt=t_max / n_steps, n_steps=n_steps)

ecc = no_kicks.eccentricity().value
fig, ax = plt.subplots(figsize=(12, 8))
ax.hist(ecc,bins=10)
ax.set_xlabel(r"Galactic  eccentricity")
ax.set_xlim(0,1)

In [ ]:
# ensemble of plots with unperturbed orbits
fig, ax = plt.subplots(figsize=(12, 8))

# plot the sun
sun_w0 = gd.PhaseSpacePosition( pos=[-8.2, 0, 0.02] * u.kpc,
                                vel=galcen_frame.galcen_v_sun )
sun_orbit = pot.integrate_orbit(sun_w0, t=orbits.t)
_ = sun_orbit.cylindrical.plot(['rho', 'z'], axes=[ax], color='black', zorder=10, alpha=0.5)

xlim = ax.get_xlim()
ylim = ax.get_ylim()

# plot the orbits with kicks
for i in range(no_kicks.shape[1]):
    _ = no_kicks[:, i].cylindrical.plot(['rho', 'z'], axes=[ax], label=r"$v_{r}="+f"{rvs[i]:.0f}"+r"$", zorder=no_kicks.shape[1]-i, alpha=0.5, linestyle=":")
# _ = no_kicks.cylindrical.plot(['rho', 'z'], axes=[ax], alpha=0.5) #, color='tab:purple', alpha=0.5)


ax.set_xlim(2.5, 12.5)
ax.set_ylim(-3,3)

ax.plot(np.nan, np.nan, color='k', label=r"$\odot$")
ax.legend(frameon=True, loc="lower left", ncol=2)
